# Lendo os dados

In [1]:

import pandas as pd
from datetime import datetime

wlcota_df = pd.read_csv(
    "https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")

# Removing some columns
wlcota_columns = wlcota_df.columns.tolist()
rm_columns = ["deaths_per_100k_inhabitants", "totalCases_per_100k_inhabitants", "deathsMS",
              "deaths_by_totalCases", "tests_per_100k_inhabitants", "totalCasesMS", "tests"]
for col in rm_columns:
    wlcota_columns.pop(wlcota_columns.index(col))
wlcota_df = wlcota_df[wlcota_columns]

wlcota_df.head()

,date,country,state,city,newDeaths,deaths,newCases,totalCases,recovered,suspects
0,2020-02-25,Brazil,SP,TOTAL,0,0,1,1,NaN,NaN
1,2020-02-25,Brazil,TOTAL,TOTAL,0,0,1,1,NaN,NaN
2,2020-02-26,Brazil,SP,TOTAL,0,0,0,1,NaN,NaN
3,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,1,NaN,NaN
4,2020-02-27,Brazil,SP,TOTAL,0,0,0,1,NaN,NaN


In [9]:
states = wlcota_df["state"].unique().tolist()
states.pop(states.index("TOTAL"))

import plotly.graph_objects as go

fig = go.Figure()
for state in states:
    state_df = wlcota_df.where(wlcota_df["state"]==state).dropna()
    fig.add_trace(go.Scatter(
        name=state,
        x=state_df["date"],
        y=state_df["recovered"],
        mode='lines',
        line_shape='spline'))
    
fig.update_layout(template='plotly_dark',
                  xaxis_title='Data', yaxis_title='Indivíduos',
                  title_text="Recuperados por estado do Brasil")
fig.show()

# Recuperando o modelo no GCloud

In [3]:

import pandas_gbq
from google.oauth2 import service_account

# The contents IDs
PROJECT_ID    = "epidemicapp-280600"
PRED_TABLE_ID = "countries.last_predictions"
# The GCloud project credentials
CREDENTIALS   = service_account.Credentials.from_service_account_file(
    './app/gkeys/epidemicapp-62d0d471b86f.json')
pandas_gbq.context.credentials = CREDENTIALS

# The query to access the 
# parameters data
query = """
    select *
    from countries.parameters
    where country = "BR"
"""

# Fetch the data from the GCloud platform
df = pandas_gbq.read_gbq(query, project_id=PROJECT_ID)
df = df.sort_values(by=["date"])

# Get the last estimated parameters for Brazil
last_parameters = df.where(df["date"] == df["date"].max()).dropna()
Ro  = last_parameters["Ro"].iloc[0] 
D   = last_parameters["D"].iloc[0]
pop = last_parameters["pop"].iloc[0]

df.head()

Downloading: 100%|██████████| 73/73 [00:01<00:00, 60.34rows/s]


,Ro,D,pop,date,country,peak_est
0,6.892396,40.0,0.05,2020-04-06 00:00:00+00:00,BR,2020-06-07 00:00:00+00:00
1,6.602191,40.0,0.05,2020-04-07 00:00:00+00:00,BR,2020-06-10 00:00:00+00:00
2,6.502439,40.0,0.05,2020-04-08 00:00:00+00:00,BR,2020-06-11 00:00:00+00:00
3,6.478179,40.0,0.05,2020-04-09 00:00:00+00:00,BR,2020-06-11 00:00:00+00:00
4,6.427373,40.0,0.05,2020-04-10 00:00:00+00:00,BR,2020-06-12 00:00:00+00:00


# Testando o modelo do país para um estado

In [4]:

state = "SP"
start_number = 1000

state_df = wlcota_df.where(wlcota_df["state"]==state).dropna()
state_df = state_df.where(state_df["totalCases"] >= start_number).dropna()
state_df['date'] = [datetime.fromisoformat(f) for f in state_df['date']]

state_df.head()

,date,country,state,city,newDeaths,deaths,newCases,totalCases,recovered,suspects
403,2020-03-26,Brazil,SP,TOTAL,10.0,58.0,191.0,1053.0,1.0,14312.0
431,2020-03-27,Brazil,SP,TOTAL,10.0,68.0,170.0,1223.0,1.0,14312.0
459,2020-03-28,Brazil,SP,TOTAL,16.0,84.0,183.0,1406.0,1.0,14312.0
487,2020-03-29,Brazil,SP,TOTAL,14.0,98.0,45.0,1451.0,1.0,14312.0
515,2020-03-30,Brazil,SP,TOTAL,15.0,113.0,66.0,1517.0,1.0,12000.0


In [5]:

import numpy as np
from models   import *
from datetime import timedelta

state_pop = 44.04e6 #SP #15.13e6 #BA #16.46e6 #RJ

# Create the SIR epidemic model
country_model = ss.SIR(pop=state_pop, focus=["S","I","R"], forced_search_pop=True)

# Create the simulation data
t_pred = np.linspace(0, 120, 121)
I0, R0 = state_df["totalCases"].iloc[0], state_df["recovered"].iloc[0] 
S0 = state_pop - I0 - R0

# Define the model parameters
country_model.parameters = (Ro, D, pop)

# Simulate the model
y_pred = country_model.predict((S0, I0, R0), t_pred)

# Create the dates of the predictions
date_pred = [state_df["date"].iloc[0] + timedelta(days=i) for i in t_pred]

# Computing the acumlated cases
nc_pred = np.diff(y_pred[1]) + np.diff(y_pred[2])

cases_total_pred = [I0]
for nc in nc_pred:
    cases_total_pred.append(cases_total_pred[-1] + nc)

Loading BokehJS ...

In [8]:

fig = go.Figure()

fig.add_trace(go.Scatter(
    name="Recuperados",
    x=state_df["date"],
    y=state_df["recovered"]))
fig.add_trace(go.Scatter(
    name="Casos Totais",
    x=state_df["date"],
    y=state_df["totalCases"]))

fig.add_trace(go.Scatter(
    name="Casos Totais Modelo",
    x=date_pred,
    y=cases_total_pred))
fig.add_trace(go.Scatter(
    name="Recuperados Modelo",
    x=date_pred,
    y=y_pred[2]))

fig.update_layout(template='plotly_dark',
                  xaxis_title='Data', yaxis_title='Indivíduos',
                  title_text="Modelo do Brasil em SP")
fig.show()